### Now we will take our project from Day 1 to the next level.

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.


In [2]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    print("API key looks good so far")
else:
    print(
        "There might be a problem with your API key? Please visit the troubleshooting notebook!"
    )

MODEL = "gpt-4o-mini"
openai = OpenAI()

API key looks good so far


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
website = Website("https://openai.com")
website.links

['#main',
 '/',
 'https://chatgpt.com/?openaicom-did=ae870901-8ab8-4bcc-b891-a22cede1671b&openaicom_referred=true',
 'https://sora.com/',
 'https://platform.openai.com/',
 '/research/index/',
 '/safety/',
 '/business/',
 '/api/',
 'https://chatgpt.com/overview?openaicom-did=ae870901-8ab8-4bcc-b891-a22cede1671b&openaicom_referred=true',
 '/sora/',
 '/stories/',
 '/about/',
 '/news/',
 '/research/index/',
 '/research/',
 '/residency/',
 '/index/introducing-gpt-5/',
 '/index/introducing-o3-and-o4-mini/',
 '/index/introducing-gpt-4-5/',
 '/o1/',
 '/index/gpt-4o-system-card/',
 '/index/sora-system-card/',
 '/safety/',
 '/security-and-privacy/',
 '/business/',
 '/business/',
 '/startups/',
 '/solutions/',
 '/business/chatgpt-pricing/',
 '/api/pricing/',
 '/contact-sales/',
 '/api/',
 '/api/pricing/',
 '/codex/',
 '/open-models/',
 'https://community.openai.com/',
 'https://chatgpt.com/overview?openaicom-did=ae870901-8ab8-4bcc-b891-a22cede1671b&openaicom_referred=true',
 'https://chatgpt.com/

### First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.


In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(website))

Here is the list of links on the website of https://openai.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main
/
https://chatgpt.com/?openaicom-did=ae870901-8ab8-4bcc-b891-a22cede1671b&openaicom_referred=true
https://sora.com/
https://platform.openai.com/
/research/index/
/safety/
/business/
/api/
https://chatgpt.com/overview?openaicom-did=ae870901-8ab8-4bcc-b891-a22cede1671b&openaicom_referred=true
/sora/
/stories/
/about/
/news/
/research/index/
/research/
/residency/
/index/introducing-gpt-5/
/index/introducing-o3-and-o4-mini/
/index/introducing-gpt-4-5/
/o1/
/index/gpt-4o-system-card/
/index/sora-system-card/
/safety/
/security-and-privacy/
/business/
/business/
/startups/
/solutions/
/business/chatgpt-pricing/
/api/pricing/
/contact-sales/
/api/
/api/pricing/
/codex/
/open-models/
https://communit

In [14]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        response_format={"type": "json_object"},
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/HunyuanImage-3.0',
 '/tencent/Hunyuan3D-Part',
 '/deepseek-ai/DeepSeek-V3.2-Exp',
 '/zai-org/GLM-4.6',
 '/Qwen/Qwen3-Omni-30B-A3B-Instruct',
 '/models',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/Qwen/Qwen-Image-Edit-2509',
 '/spaces/multimodalart/ai-toolkit',
 '/spaces',
 '/datasets/openai/gdpval',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Nemotron-Personas-Japan',
 '/datasets/t-tech/T-ECD',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/d

In [16]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

### Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o


In [24]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanImage-3.0
Updated
about 24 hours ago
•
548
•
696
tencent/Hunyuan3D-Part
Up

In [26]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanImage-3.0\nUpdated\nabout 24 hours ago\n•\n548\n•\n696\ntencent/Hunyuan3D-Part\nUpdated\n3 days ago\n•\n2.33k\n•\n451\ndeepseek-ai/DeepSeek-V3.2-Exp\nUpdated\n2 days ago\n•\n5.6k\n•\n433\nzai-org/GLM-4.6\nUpdated\n1 day ago\n•\n222\nQwen/Qwen3-Omni-30B-A3B-Instruct\nUpdated\n9 days ago\n•\n152k\n•\n567\nBrowse 1M+ models\nSpaces\nRunning\n1.24k\n1.24k\nWan2.2 Animate\n👁\nWan

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'company GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'company Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'company LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Building the Future of AI Together

---

### About Us
Hugging Face is an innovative platform at the forefront of the AI and machine learning community. Our goal is to create a collaborative environment where professionals and enthusiasts can build, discover, and share state-of-the-art models, datasets, and applications. With a commitment to open-source technology, Hugging Face aims to make machine learning accessible for everyone, helping propel the industry forward in an inclusive and community-driven manner.

---

### Our Mission
We believe in the power of collaboration and community. At Hugging Face, we’re building a future where machine learning is more accessible, efficient, and impactful. Our platform serves as the home for over 1 million models and provides numerous datasets and tools for developers and researchers to optimize their work in AI.

---

### Platform Overview
- **Models:** Browse and deploy a vast array of AI models, including state-of-the-art options for various modalities like text, images, video, and audio.
- **Datasets:** Access over 250,000 publicly available datasets essential for training robust machine learning models.
- **Spaces:** Utilize our collaborative spaces for running applications, exploring AI functionalities, or showcasing models with our community.
  
---

### Our Community
With over 50,000 organizations using Hugging Face, we proudly host contributions from prominent leaders in tech, including:
- **Meta**
- **Google**
- **Amazon**
- **Microsoft**
  
These companies utilize our platform to accelerate their AI projects, tapping into the collective intelligence and resources of our broad community.

---

### Company Culture
At Hugging Face, our culture is steeped in open collaboration and innovation. We prioritize supportive teamwork, foster creativity, and embrace diverse perspectives. Each team member is empowered to contribute ideas, and we celebrate the successes of our community as a shared accomplishment.

### Careers at Hugging Face
We are continuously looking for passionate individuals to join our team. Opportunities abound in various fields from engineering and research to community engagement and customer support. Embrace the chance to work in a dynamic environment where you can impact the future of AI.

- **Why Join Us?**
  - Be part of a pioneering company focused on shaping the landscape of artificial intelligence.
  - Work with leaders in the industry and the brightest minds in technology.
  - Contribute to open-source projects that will help millions of users worldwide.

---

### Get Involved
If you're passionate about AI and want to contribute to a vibrant community or explore opportunities with us:
- **Explore Models and Datasets:** [Hugging Face Hub](https://huggingface.co)
- **Join our Team:** Check out current job openings on our [Careers Page](https://huggingface.co/jobs).
- **Connect with Us:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), or join our [Discord](https://discord.gg/huggingface) community.

---

Together, let's build the future of artificial intelligence!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation


In [34]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## The AI Community Building the Future

Welcome to **Hugging Face**, the leading platform where the machine learning community collaborates to create models, datasets, and applications.

---

### About Us

At Hugging Face, we are dedicated to fostering an open and collaborative environment for machine learning enthusiasts and professionals. Our goal is to transform how people engage with AI by providing state-of-the-art tools, resources, and a community-centric approach. 

---

### Our Offerings

- **Models**: Access over **1 million** pre-trained models to jumpstart your projects.
- **Datasets**: Explore more than **250,000 datasets** to train your models efficiently.
- **Spaces**: Develop and deploy over **400,000 applications** seamlessly.
- **Enterprise Solutions**: Tailor-made services for organizations, including advanced security and dedicated support.

---

### Our Community

We bring together over **50,000** organizations, including:

- **Meta**
- **Microsoft**
- **Amazon**
- **Google**

Join a vibrant community and collaborate with top-tier companies and researchers alike.

---

### Company Culture

Hugging Face thrives on inclusivity, transparency, and collaboration. We believe that the best innovations arise from diverse perspectives and shared knowledge. This ethos drives our community and product development. 

We encourage all team members to **contribute** and be active participants in shaping the future of AI through our open-source initiatives.

---

### Careers at Hugging Face

Join a passionate team of individuals dedicated to pushing the boundaries of AI. We’re on the lookout for talent across various domains:

- Research Scientists
- Machine Learning Engineers
- Community Managers
- Software Developers

**Perks of Working With Us:**

- Work in a **remote-first** environment.
- Engage in continuous learning and professional development.
- Collaborate with global AI experts and enthusiasts.

---

### Getting Started

Whether you are looking to explore AI applications, leverage our extensive catalog of models and datasets, or become a part of our growing team, **Hugging Face** is here for you. 

**Sign Up** today or learn more about our [Enterprise Solutions](https://huggingface.co/enterprise) to accelerate your machine learning journey.

---

Join us as we build the future of AI together! 

[Visit our website](https://huggingface.co) for more information.